In [22]:
import pandas as pd 
import ImbalancedLearningRegression as iblr
from xgboost import XGBRFRegressor, XGBRegressor
from lightgbm import LGBMRegressor

df = pd.read_csv('data.csv')
df.head()

,index,rdkit_smi,herg_pIC50,hepg2_pIC50,cyp1_pIC50,cyp2_pIC50,cyp3_pIC50,rdkit0,rdkit1,rdkit2,...,ChemBERTa758,ChemBERTa759,ChemBERTa760,ChemBERTa761,ChemBERTa762,ChemBERTa763,ChemBERTa764,ChemBERTa765,ChemBERTa766,ChemBERTa767
0,0,Br.CCCCOc1ccc2c3ccnc(C)c3n(CC(C)C)c2c1,NaN,-4.082785,NaN,NaN,NaN,2.142479,-2.118959,2.328215,...,0.180178,-0.185827,-1.386276,0.212496,-0.035696,1.496147,0.087391,-0.816885,-0.166529,-0.102956
1,1,Br.CCCCOc1ccc2c3ccnc(C)c3n(CCCC)c2c1,NaN,-4.190332,NaN,NaN,NaN,2.131358,-2.092394,2.325443,...,0.131249,-0.237652,-1.550219,0.256954,-0.175876,1.530199,0.007508,-0.706796,0.077539,-0.224275
2,2,Br.CCCCn1c2cc(O)ccc2c2ccnc(C)c21,NaN,-4.910000,NaN,NaN,NaN,2.124477,-2.073277,2.319935,...,0.424615,-0.148931,-1.203021,0.256493,-0.045351,1.712546,0.162551,-0.709963,0.325040,-0.077512
3,3,Br.CCCCn1c2cc(OC(C)C)ccc2c2ccnc(C)c21,NaN,-4.480000,NaN,NaN,NaN,2.134342,-2.111540,2.326932,...,0.139105,0.165535,-1.101270,0.166301,-0.274676,1.776415,0.096857,-1.068090,0.204451,-0.077804
4,4,Br.CCCCn1c2cc(OCC)ccc2c2ccnc(C)c21,NaN,-4.640000,NaN,NaN,NaN,2.129806,-2.085897,2.324375,...,0.197084,-0.165342,-1.175774,0.275327,-0.278163,1.767010,0.078930,-0.908876,0.130458,-0.116104


In [23]:
# import deepchem as dc

# rdkit_featurizer = dc.feat.RDKitDescriptors()
# features = rdkit_featurizer.featurize(df['rdkit_smi'])

# features = pd.DataFrame(features)

# features = features.reset_index(drop=True) # 분자의 화학적 특성 df
# features.columns = ['rdkit'+str(i) for i in range(features.shape[1])]

# df = df.reset_index()
# features['rdkit_smi'] = df['rdkit_smi']
# df = pd.merge(df,features)

# featurizer = dc.feat.CircularFingerprint(size=2048, radius=4)
# total_fp_sum = pd.DataFrame([[sum(featurizer.featurize(smiles)[0]),smiles] for smiles in df['rdkit_smi']],columns=['fp_sum','rdkit_smi'])

# df = pd.merge(df,total_fp_sum)

# df.to_csv('data.csv',index=False)

In [24]:
# from transformers import AutoTokenizer, AutoModel
# import torch

# # ChemBERTa 모델과 토크나이저 로드
# model_name = 'seyonec/ChemBERTa-zinc250k-v1'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# def ChemBERTa_feature(smiles):

#     inputs = tokenizer(smiles, return_tensors='pt', padding=True, truncation=True)

#     with torch.no_grad():
#         outputs = model(**inputs) #

#     feature_vector = outputs.last_hidden_state[:, 0, :].numpy()
#     # 분자 특성 벡터 추출
#     return list(feature_vector.reshape(-1,))

# total_ChemBERTa_df = pd.DataFrame([ChemBERTa_feature(i)+[i] for i in df['rdkit_smi']])
# total_ChemBERTa_df.columns = ['ChemBERTa' + str(i) for i in total_ChemBERTa_df.columns[:-1]] + ['rdkit_smi']

# df = pd.merge(df, total_ChemBERTa_df)
# df.shape

In [25]:
import numpy as np
# inf, NaN, 범위 초과 모두 확인
for_pca_df = df[[i for i in df.columns if 'pIC50' not in i and i not in ['rdkit_smi','index']]]
for_pca_df = for_pca_df.fillna(0)

# mask = (np.isinf(for_pca_df)) | (np.isnan(for_pca_df)) | (np.abs(for_pca_df) > 1e308) # inf값 존재
# remove_idx = for_pca_df[mask.any(axis=1)].index

# df = df.drop(remove_idx)
# for_pca_df = for_pca_df.drop(remove_idx)
for_pca_df = for_pca_df.reset_index(drop=True)

In [26]:
# 현제 data.csv는 rdkitdescriptor, chemberta까지 끝낸상태

In [28]:
from sklearn.decomposition import PCA

# 몇 천개의 특성을 pca로 줄임
pca = PCA(n_components=0.999)  
df_pca = pca.fit_transform(for_pca_df)

df_pca = pd.DataFrame(df_pca).reset_index(drop=True)
df = df.reset_index(drop=True)

pIC_list = ['herg_pIC50', 'hepg2_pIC50', 'cyp1_pIC50', 'cyp2_pIC50', 'cyp3_pIC50']
df_pca[pIC_list] = df[pIC_list]

df_pca.columns = ['pca'+str(i) if '_' not in str(i) else str(i) for i in df_pca.columns]

In [30]:
# https://github.com/YyzHarry/imbalanced-regression
# sts-b-dir 폴더 git clone 하기!
# sts-b-dir 폴더 내에 util.py가 있는데 이거 이름을 utils.py로 바꿔야 밑에 함수가 불러와짐
import sys
from collections import Counter
from scipy.ndimage import convolve1d

sys.path.append(r'D:\\toxic_prediction\\sts-b-dir') # LDS를 위한 함수를 불러오기 위해서

from utils import get_lds_kernel_window

def get_bin_idx(label, num_bins, label_min, label_max):
    if label_max == label_min:
        return 0
    
    normalized = (label - label_min) / (label_max - label_min)
    bin_idx = int(normalized * num_bins)   
    return max(0, min(num_bins - 1, bin_idx))

def lds(labels):
    # preds, labels: [Ns,], "Ns" is the number of total samples
    # assign each label to its corresponding bin (start from 0)
    # with your defined get_bin_idx(), return bin_index_per_label: [Ns,] 
    label_min = min(labels)
    label_max = max(labels)
    num_bins = 50

    bin_index_per_label = [get_bin_idx(label, num_bins, label_min, label_max) for label in labels]

    # calculate empirical (original) label distribution: [Nb,]
    # "Nb" is the number of bins
    Nb = max(bin_index_per_label) + 1
    num_samples_of_bins = dict(Counter(bin_index_per_label))
    emp_label_dist = [num_samples_of_bins.get(i, 0) for i in range(Nb)]

    # lds_kernel_window: [ks,], here for example, we use gaussian, ks=5, sigma=2
    lds_kernel_window = get_lds_kernel_window(kernel='gaussian', ks=5, sigma=2)
    # calculate effective label distribution: [Nb,]
    eff_label_dist = convolve1d(np.array(emp_label_dist), weights=lds_kernel_window, mode='constant')

    # Use re-weighting based on effective label distribution, sample-wise weights: [Ns,]
    eff_num_per_label = [eff_label_dist[bin_idx] for bin_idx in bin_index_per_label]
    lds_pIC50 = [np.float32(1 / x) for x in eff_num_per_label]
    lds_pIC50 = lds_pIC50 / np.mean(lds_pIC50) # 원본 코드에는 이게 없는데 이렇게 해야 가중치 간 값의 차이가 좀 생김
    # 이거 안하면 0.62, 0.0054 이런식이라 모델이 제대로 가중치 적용을 못하는듯 하여 이렇게 한 줄 추가함

    return lds_pIC50

In [31]:
# 가우시안 노이즈로 증강한 데이터를 가지고 모델 학습 후 CAS_KPBMA_MAP3K5_IC50s.xlsx의 레이블 추가하기 
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score

model = LGBMRegressor(
            random_state=42, n_estimators=500, min_child_weight=5, n_jobs=-1,
            learning_rate = 0.05
        )

def kfold(model, X_resampled, y_resampled):

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    mse_list, rmse_list, r2_list = [], [], []

    # 인덱스 리셋(정렬/정합 문제 방지)
    Xr = X_resampled.reset_index(drop=True)
    yr = y_resampled.reset_index(drop=True)
    w_all = np.asarray(lds(yr), dtype=float)  # 전체에서 만든 가중치라면

    for tr_idx, te_idx in kf.split(Xr):
        X_train, X_test = Xr.iloc[tr_idx], Xr.iloc[te_idx]
        y_train, y_test = yr.iloc[tr_idx], yr.iloc[te_idx]

        weight = w_all[tr_idx]                       
        model.fit(X_train, y_train, sample_weight=weight)

        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        mse_list.append(mse)
        rmse_list.append(rmse)
        r2_list.append(r2)

    print("각 Fold의 MSE:", mse_list)
    print("평균 MSE:", np.mean(mse_list))
    print("각 Fold의 RMSE:", rmse_list)
    print("평균 RMSE:", np.mean(rmse_list))
    print("각 Fold의 R2:", r2_list)
    print("평균 R2:", np.mean(r2_list))

In [32]:
def for_ml_train_data(pIC50_name):
	train = df_pca[~df_pca[pIC50_name].isnull()]
	train = train[[i for i in train.columns if 'pIC50' not in i] + [pIC50_name]]
	return train

pIC_list = ['herg_pIC50', 'hepg2_pIC50', 'cyp1_pIC50', 'cyp2_pIC50', 'cyp3_pIC50']

for y_name in pIC_list:
	pIC50_name = y_name

	train_data = for_ml_train_data(pIC50_name)
	train_data = train_data.reset_index(drop=True)

	df_gn = iblr.gn(
	data=train_data, y=y_name)

	X_resampled = df_gn.drop(columns=[y_name])
	y_resampled = df_gn[y_name]
											
	print(y_name)
	kfold(model, X_resampled, y_resampled)

	df_pca = df_pca.reset_index(drop=True)
	df = df.reset_index(drop=True)

	pred = model.predict(df_pca[df_pca[y_name].isnull()][[i for i in df_pca.columns if 'pca' in i]])
	df.loc[df[y_name].isnull(), y_name] = pred

r_index: 100%|##########| 1499/1499 [00:55<00:00, 26.91it/s]


herg_pIC50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003849 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 9612, number of used features: 35
[LightGBM] [Info] Start training from score -1.650824
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 9613, number of used features: 35
[LightGBM] [Info] Start training from score -1.613736
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003698 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 9613, number of used features: 35
[LightGBM] [Info]

r_index: 100%|##########| 1941/1941 [01:47<00:00, 18.03it/s]


hepg2_pIC50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 18860, number of used features: 35
[LightGBM] [Info] Start training from score -1.638381
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006141 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 18860, number of used features: 35
[LightGBM] [Info] Start training from score -1.415835
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005714 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 18860, number of used features: 35
[LightGBM] [I

r_index: 100%|##########| 128/128 [00:02<00:00, 42.89it/s]


cyp1_pIC50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001466 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 4249, number of used features: 35
[LightGBM] [Info] Start training from score -2.379071
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001485 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 4249, number of used features: 35
[LightGBM] [Info] Start training from score -2.280384
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001417 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 4250, number of used features: 35
[LightGBM] [Info]

r_index: 100%|##########| 246/246 [00:06<00:00, 35.78it/s]


cyp2_pIC50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003074 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 4838, number of used features: 35
[LightGBM] [Info] Start training from score -1.489119
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001552 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 4838, number of used features: 35
[LightGBM] [Info] Start training from score -1.732543
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001531 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 4838, number of used features: 35
[LightGBM] [Info]

r_index: 100%|##########| 353/353 [00:13<00:00, 26.09it/s]


cyp3_pIC50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002983 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 7160, number of used features: 35
[LightGBM] [Info] Start training from score -1.880693
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002561 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 7161, number of used features: 35
[LightGBM] [Info] Start training from score -1.701987
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002845 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8925
[LightGBM] [Info] Number of data points in the train set: 7161, number of used features: 35
[LightGBM] [Info]

In [38]:
df = df.fillna(0)

In [40]:
df.to_csv('data.csv',index=False)

In [41]:
# herg_pIC50
# 평균 MSE: 0.3548311469286793
# 평균 RMSE: 0.5956003594530906
# 평균 R2: 0.9055410523424247

# hepg2_pIC50
# 평균 MSE: 0.4615955993274028
# 평균 RMSE: 0.6793576153288241
# 평균 R2: 0.8846750312159586

# cyp1_pIC50
# 평균 MSE: 0.2626510742523773
# 평균 RMSE: 0.5116629978106569
# 평균 R2: 0.9328306967726728

# cyp2_pIC50
# 평균 MSE: 0.3100971827804355
# 평균 RMSE: 0.5566655167679544
# 평균 R2: 0.9185350089672788

# cyp3_pIC50
# 평균 MSE: 0.36290323937124197
# 평균 RMSE: 0.6020723315551127
# 평균 R2: 0.902082913142169